In [ ]:
# 多组测试数据相关性表现：[1.0, 0.9999999999999999, 0.9999999999999999]

In [1]:
# 设定所有月份的oneminute数据经过factor_pycode处理后将要保存的位置

######################################################################### factor_result_allmonth路径替换
output_directory = r'F:\QuantData\factor_result_allmonth\factor_47_WeightedSkewness'
######################################################################### factor_result_allmonth路径替换

In [2]:
# 定义函数get_factor.给定分钟数据，返回形如'SecuCode''TradeDay'因子值三列的结果数据
# 取自 20047_3.py
import pandas as pd
import numpy as np

def get_factor(data): ### df数据表参量名替换
    ################################################################################################# factor_pycode代码替换
    # 计算每个交易日的加权偏度
    def calculate_weighted_skewness(df):
        # 计算总成交量
        total_vol = df['Volume'].sum()
        
        # 计算权重
        df['w_t'] = df['Volume'] / total_vol
        
        # 计算收盘价均值
        mean_close = df['ClosePrice'].mean()
        
        # 计算收盘价标准差
        std_close = df['ClosePrice'].std()
        
        # 计算每个时间段的偏度贡献
        df['skew_contribution'] = df['w_t'] * ((df['ClosePrice'] - mean_close) ** 3)
        
        # 计算总偏度贡献
        total_skew_contribution = df['skew_contribution'].sum()
        
        # 计算加权偏度
        weighted_skewness = total_skew_contribution / (std_close ** 3)
        
        return weighted_skewness
    
    # 分组计算每只股票每个交易日的加权偏度
    result = data.groupby(['SecuCode', 'TradeDay']).apply(calculate_weighted_skewness).reset_index()
    result.columns = ['SecuCode', 'TradeDay', 'WeightedSkewness']
    ################################################################################################# factor_pycode代码替换

    
    ############################################################### result结果数据表命名替换  &  因子列命名替换    
    result.rename(columns={'WeightedSkewness': 'FactorValue'}, inplace=True)
    ############################################################### result结果数据表命名替换  &  因子列命名替换    
    
    ################ result结果数据表命名替换
    return result
    ################ result结果数据表命名替换 

In [3]:
# 导入所有one_minute文件，生成相对应factor_result文件，并保存（每个文件对应某年某月）

import pandas as pd
import os

def generate_factor_files(input_directory, output_directory, process_function, start_year, start_month, end_year, end_month):
    # 生成所有的文件名，按年月顺序
    input_file_names = []

    # 生成文件名列表，考虑到起始年月和结束年月的数据
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if (year == start_year and month < start_month) or (year == end_year and month > end_month):
                continue
            file_name = f"{year}{str(month).zfill(2)}_oneminute.feather"
            input_file_names.append(file_name)

    # 创建输出目录（如果不存在）
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 处理并保存数据
    for file in input_file_names:
        input_file_path = os.path.join(input_directory, file)
        if os.path.exists(input_file_path):  # 检查文件是否存在
            # 读取Feather文件
            df_oneminute = pd.read_feather(input_file_path)
            
            # 调用数据处理函数
            df_res = process_function(df_oneminute)
            
            # 定义输出文件名
            year_month = file[:6]  # 提取文件名前6个字符作为年月
            output_file_name = f"{year_month}_factor_result.feather"
            output_file_path = os.path.join(output_directory, output_file_name)
            
            # 保存处理后的DataFrame为Feather文件
            df_res.to_feather(output_file_path)
            
            # print(f"Processed and saved {output_file_path}")
        else:
            print(f"File {file} does not exist.")

# one_minute数据存放的位置
input_directory = r'F:\QuantData\minute_data'

# 起始年月和结束年月（回测区间）
start_year, start_month = 2015, 1
end_year, end_month = 2024, 7

# 生成所有factor_result文件
generate_factor_files(input_directory, output_directory, get_factor, start_year, start_month, end_year, end_month)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12932\3586327628.py:29: RuntimeWarning: invalid value encountered in scalar divide
  weighted_skewness = total_skew_contribution / (std_close ** 3)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12932\3586327628.py:29: RuntimeWarning: invalid value encountered in scalar divide
  weighted_skewness = total_skew_contribution / (std_close ** 3)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12932\3586327628.py:29: RuntimeWarning: invalid value encountered in scalar divide
  weighted_skewness = total_skew_contribution / (std_close ** 3)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12932\3586327628.py:29: RuntimeWarning: invalid value encountered in scalar divide
  weighted_skewness = total_skew_contribution / (std_close ** 3)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12932\3586327628.py:29: RuntimeWarning: invalid value encountered in scalar divide
  weighted_skewness = total_skew_contribution / (std_close ** 3)


In [2]:
# 每日高开低收数据df_Day_allmonth（201501-202407）把前面计算得到的factor_result数据全部整合进去

import pandas as pd
import numpy as np
import os

df_Day_allmonth = pd.read_feather('F:\\QuantData\\AShareEODPrices_allmonth.feather')

# 确保 TradeDay 列转换为日期时间格式
df_Day_allmonth['TradeDay'] = pd.to_datetime(df_Day_allmonth['TradeDay'])

output_file_names = [f for f in os.listdir(output_directory) if f.endswith('_factor_result.feather')]

# 先合并所有factor_result文件，再与df_Day_allmonth合并
list_res = []
for file in output_file_names:
    output_file_path = os.path.join(output_directory, file)
    
    # 读取Feather文件
    df_res = pd.read_feather(output_file_path)
    list_res.append(df_res)

df_factor_allmonth = pd.concat(list_res)

# 确保 TradeDay 列转换为日期时间格式
df_factor_allmonth['TradeDay'] = pd.to_datetime(df_factor_allmonth['TradeDay'])

# 与df_Day_allmonth合并
df_merge = pd.merge(df_factor_allmonth, df_Day_allmonth, on=['SecuCode', 'TradeDay'], how='left')


In [3]:
# 对每个交易日进行横截面市值中性化处理（消除市值对因子值的影响）

import statsmodels.api as sm

def neutralize_by_mv(df, factor):
    df = df.copy()
    df['logMV'] = np.log(df['TotalMV'].replace(0, np.nan))  # 避免log(0)带来的问题
    neutralized_values = []

    for date, group in df.groupby('TradeDay'):
        group = group.dropna(subset=[factor, 'logMV']).copy()  # 确保 group 是一个副本
        if group.empty:
            continue
        y = group[factor]
        X = sm.add_constant(group['logMV'])
        if len(X) > 1:  # 确保有足够的数据点进行回归
            model = sm.OLS(y, X).fit()
            group[factor + '_neu'] = model.resid
        else:
            group[factor + '_neu'] = np.nan
        neutralized_values.append(group)

    return pd.concat(neutralized_values)
    
# FactorValue经上述中性化操作得到FactorValue_neu
df_PT = neutralize_by_mv(df_merge, 'FactorValue')


C:\Users\Administrator\anaconda3\envs\quant\Lib\site-packages\statsmodels\regression\linear_model.py:1705: RuntimeWarning: invalid value encountered in subtract
  return self.model.endog - self.model.predict(
C:\Users\Administrator\anaconda3\envs\quant\Lib\site-packages\statsmodels\regression\linear_model.py:1705: RuntimeWarning: invalid value encountered in subtract
  return self.model.endog - self.model.predict(
C:\Users\Administrator\anaconda3\envs\quant\Lib\site-packages\statsmodels\regression\linear_model.py:1705: RuntimeWarning: invalid value encountered in subtract
  return self.model.endog - self.model.predict(
C:\Users\Administrator\anaconda3\envs\quant\Lib\site-packages\statsmodels\regression\linear_model.py:1705: RuntimeWarning: invalid value encountered in subtract
  return self.model.endog - self.model.predict(


In [5]:
df_PT

,SecuCode,TradeDay,FactorValue,SecuName,PreClosePrice,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,...,TotalMV,IfQ4LowCover,IfReportPeriodLate,IfTrade,IfTrade.Tag,logMV,FactorValue_neu,Return,Month,logReturn
0,000001.SZ,2015-01-05,0.626612,平安银行,15.84,15.99,16.28,15.60,16.02,286043643.0,...,1.830268e+11,0.0,0.0,0.0,",03",25.932899,0.278511,0.011364,2015-01,0.011300
20,000002.SZ,2015-01-05,-3.952918,万科A,13.90,14.39,15.29,14.22,14.91,656083570.0,...,1.642341e+11,0.0,0.0,1.0,,25.824558,-4.262765,0.072662,2015-01,0.070143
40,000004.SZ,2015-01-05,1.279667,国华网安,15.57,15.58,15.90,15.30,15.69,2809512.0,...,1.317594e+09,1.0,0.0,0.0,",06",20.999073,2.673686,0.007707,2015-01,0.007678
60,000006.SZ,2015-01-05,3.133380,深振业A,7.05,7.10,7.39,7.02,7.08,107864712.0,...,9.557965e+09,1.0,0.0,0.0,",06",22.980641,3.827712,0.004255,2015-01,0.004246
80,000007.SZ,2015-01-05,0.686835,*ST全新,14.60,14.51,14.80,14.25,14.52,2841572.0,...,3.353617e+09,1.0,0.0,0.0,",06",21.933305,1.750979,-0.005479,2015-01,-0.005495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917441,688799.SH,2024-07-22,1.296934,华纳药厂,41.69,41.41,42.06,41.17,41.62,334451.0,...,3.903956e+09,0.0,0.0,1.0,,22.085256,2.461487,-0.001679,2024-07,-0.001680
8917461,688800.SH,2024-07-22,7.996441,瑞可达,25.90,26.00,26.55,25.77,25.81,1690767.0,...,4.088817e+09,0.0,0.0,1.0,,22.131522,9.101917,-0.003475,2024-07,-0.003481
8917481,688819.SH,2024-07-22,1.831254,天能股份,22.15,22.00,22.55,21.99,22.20,1730837.0,...,2.158062e+10,0.0,0.0,1.0,,23.795062,0.812507,0.002257,2024-07,0.002255
8917501,688981.SH,2024-07-22,1.429310,中芯国际,51.20,50.90,51.45,50.65,50.98,30557978.0,...,4.056137e+11,0.0,0.0,1.0,,26.728667,-3.335443,-0.004297,2024-07,-0.004306


In [4]:
# 对FactorValue_neu进行十分组多空对冲净值走势分析，绘制回测曲线并计算年化收益率等指标

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 如果分组序号与因子值呈相反关系，则需要将因子值取负，从而得到倒置的分组序号
# df_PT['FactorValue_neu'] = -df_PT['FactorValue_neu']

# 计算每支股票每天的收益率
df_PT['Return'] = df_PT['ClosePrice'] /  df_PT['PreClosePrice'] - 1

# 创建一个字典用于保存每组的净值
net_values = {i: pd.DataFrame(columns=['Month', 'NetValue']) for i in range(1, 11)}
long_short_net_value = pd.DataFrame(columns=['Month', 'NetValue'])

# 自动分析回测区间的开始和结束日期
df_PT['TradeDay'] = pd.to_datetime(df_PT['TradeDay'])
start_date = df_PT['TradeDay'].min().strftime('%Y-%m')
end_date = df_PT['TradeDay'].max().strftime('%Y-%m')

# 初始化每组的净值为1
initial_net_value = 1
for i in range(1, 11):
    net_values[i] = pd.concat([net_values[i], pd.DataFrame({'Month': [start_date], 'NetValue': [initial_net_value]})])

long_short_net_value = pd.concat([long_short_net_value, pd.DataFrame({'Month': [start_date], 'NetValue': [initial_net_value]})])

# 计算每个月的末日
df_PT['Month'] = df_PT['TradeDay'].dt.to_period('M')
end_of_month = df_PT.groupby('Month')['TradeDay'].max().reset_index()

# 新增logReturn列
df_PT['logReturn'] = np.log(df_PT['Return'] + 1)

# 循环处理每个月的数据
for i in range(len(end_of_month) - 1):
    month_end_day = end_of_month.iloc[i]['TradeDay']
    next_month_end_day = end_of_month.iloc[i + 1]['TradeDay']

    # 获取该月最后一天的所有股票数据
    month_data = df_PT[df_PT['TradeDay'] == month_end_day].copy()
    
    # 对FactorValue_neu进行十分组
    # month_data = month_data.dropna(subset=['FactorValue_neu'])
    month_data.loc[:, 'FactorValue_rank'] = pd.qcut(month_data['FactorValue_neu'], 10, labels=False) + 1

    returns = {}

    for group in range(1, 11):
        # 选出第x组的股票
        group_stocks = month_data[month_data['FactorValue_rank'] == group]['SecuCode'].values

        # 获取下个月这组股票的数据
        next_month_data = df_PT[(df_PT['TradeDay'] > month_end_day) & (df_PT['TradeDay'] <= next_month_end_day) & 
                                (df_PT['SecuCode'].isin(group_stocks))]

        # 计算每支股票的月收益率
        next_month_log_returns = next_month_data.groupby('SecuCode')['logReturn'].sum()
        next_month_returns = np.exp(next_month_log_returns) - 1

        # 计算第x组股票的平均月收益率
        avg_return = next_month_returns.mean()
        returns[group] = avg_return

        # 计算新的净值
        new_net_value = net_values[group].iloc[-1]['NetValue'] * (1 + avg_return)
        net_values[group] = pd.concat([net_values[group], pd.DataFrame({'Month': [str(next_month_end_day)[:7]], 'NetValue': [new_net_value]})])

    # 计算long-short策略的净值
    long_short_return = (1 + returns[10]) / (1 + returns[1]) - 1
    new_long_short_net_value = long_short_net_value.iloc[-1]['NetValue'] * (1 + long_short_return)
    long_short_net_value = pd.concat([long_short_net_value, pd.DataFrame({'Month': [str(next_month_end_day)[:7]], 'NetValue': [new_long_short_net_value]})])

# 绘制净值曲线
plt.figure(figsize=(12, 8))

for group in range(1, 11):
    plt.plot(net_values[group]['Month'], net_values[group]['NetValue'], linestyle='-', label=f'Group {group}')

plt.plot(long_short_net_value['Month'], long_short_net_value['NetValue'], linestyle='--', color='black', label='Long-Short')

plt.xlabel('Month')
plt.ylabel('Net Value')
plt.title('Net Value Curves for Groups 1 to 10 and Long-Short Strategy Based on FactorValue_neu')
plt.legend()

# 设置横轴每三个月标注一次
xticks = np.arange(0, len(long_short_net_value), step=3)
xlabels = [long_short_net_value['Month'].iloc[i] for i in xticks]
plt.xticks(ticks=xticks, labels=xlabels, rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()

# 计算long-short策略的绩效指标
long_short_net_value['Return'] = long_short_net_value['NetValue'].pct_change()

# 年化收益率
annual_return = (long_short_net_value['NetValue'].iloc[-1] / long_short_net_value['NetValue'].iloc[0]) ** (12 / (len(long_short_net_value) - 1)) - 1

# 年化波动率
annual_volatility = long_short_net_value['Return'].std() * np.sqrt(12)

# 信息比率
information_ratio = annual_return / annual_volatility

# 月度胜率
monthly_win_rate = (long_short_net_value['Return'] > 0).mean()

# 最大回撤率
rolling_max = long_short_net_value['NetValue'].cummax()
drawdown = (long_short_net_value['NetValue'] - rolling_max) / rolling_max
max_drawdown = drawdown.min()

# 打印结果
print(f"Annual Return: {annual_return:.2%}")
print(f"Annual Volatility: {annual_volatility:.2%}")
print(f"Information Ratio: {information_ratio:.2f}")
print(f"Monthly Win Rate: {monthly_win_rate:.2%}")
print(f"Max Drawdown: {max_drawdown:.2%}")

ValueError: Bin edges must be unique: Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype='float64', name='FactorValue_neu').
You can drop duplicate edges by setting the 'duplicates' kwarg